#  Finetune a GPT-2 Text-Generating Model with own text

based on  [Max Woolf](http://minimaxir.com) nootebook





# 1. Installing GPT-2

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
# from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [23]:
from IPython.display import HTML
from IPython.display import Image

# 2. Mounting google drive

In [2]:
#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Executing funcion to avoid disconnecting 

In [3]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


## Verifying which GPU is active 

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [4]:
!nvidia-smi

Thu Jan 21 09:57:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 3. Downloading GPT-2 if you need to fine tune

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [5]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 568Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 144Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 848Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:05, 270Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 849Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 100Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 174Mit/s]                                                       


## Txt file below is a result of concatenation the couple of books, articles on the subject which is interesting for us (in this case ~5MB management leadership, good practices in business etc.)

In [12]:
file_name = "business.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [13]:
gpt2.copy_file_from_gdrive(file_name)

# 4. Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='"355M"',
              steps=5000,
              restore_from='fresh',
              run_name='"355M"',
              print_every=10,
              sample_every=200,
              save_every=500
              )

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run355')

# 5. Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [14]:
cp /content/drive/MyDrive/GPT/checkpoint_run355.tar  /content/drive/MyDrive/

In [15]:
gpt2.copy_checkpoint_from_gdrive(run_name='run355')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
#  !rm -R /content/checkpoint/run1/

In [ ]:
cd /content

/content


In [ ]:
ls /content/checkpoint/run355/

checkpoint    events.out.tfevents.1611038631.2bacb2806925  model-5000.index
counter       hparams.json                                 model-5000.meta
encoder.json  model-5000.data-00000-of-00001               vocab.bpe


In [17]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run355')

Loading checkpoint checkpoint/run355/model-5000
INFO:tensorflow:Restoring parameters from checkpoint/run355/model-5000


# 6.Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [22]:
gpt2.generate(sess, run_name='run355')

fixed language and thinking patterns—both in the workplace and in life—can make or break a relationship. Emotional baggage—the things we have been taught in our education and from our parents—can get in the way of forging strong connections. And psychological well-being doesn’t stop at 30 days. We can linger beyond the end of a relationship or beyond a job.

If you’ve suffered a setback or had a bad day at work, you may be struggling to rebuild your relationship with them. Your manager may have told you that you need to get better at keeping your emotions in check. The fight or flight response you feel may signal that you need to examine your emotional state more closely. If it turns out that your emotional state is unhealthy, you may feel justified in staying in that position.

When we do engage in effective conflict management, we make sure we do it in a way that helps us grow. We make sure we use tools that help us learn and grow, and we make sure we practice using neutral language 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

# 7. =======================================  Let's generate some text with given start sentence/prefix

In [80]:
#@title Prefix text { display-mode: "form" }
give_run_name = 'run355' #@param ["None"] {type:"raw", allow-input: true}
paste_prefix_text = "Modern  leader knows what to do." #@param [""] {allow-input: true}
set_temperature = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.05}
set_text_length = 420 #@param {type:"slider", min:0, max:1023, step:10}
set_nr_samples = 5 #@param {type:"slider", min:4, max:10, step:1}



print("please wait text is generated..." ) 

gpt2.generate(sess,
              run_name=give_run_name,
              length=set_text_length,
              temperature=set_temperature,
              prefix=paste_prefix_text,
              nsamples=set_nr_samples,
              batch_size=set_nr_samples
              )

please wait text is generated...
Modern  leader knows what to do.





3. Collaborate and Build Team

Leaders who want to lead effective teams must be able to get along with people who are not as experienced or as smart as they are. That means making it a point to spend time with and work with people who are like themselves, and not judging them because they have different backgrounds.

In developing a team, leaders need to be careful not to make everyone feel like a member of the team. That can lead to resentment and division. Instead, create opportunities for team members to bond and develop their own ideas. That can help employees share ideas and differentiate themselves, and it will lead to more collaboration down the road.

For example, let’s say your boss expects a team member to be efficient and effective at all times. You could, for example, give her some tasks that she is not suited for and ask her to work on something that she would like to do but cannot due to a business nee

In [81]:
#@title Prefix text { display-mode: "form" }
give_run_name = 'run355' #@param ["None"] {type:"raw", allow-input: true}
paste_prefix_text = "When you don't know what to do." #@param [""] {allow-input: true}
set_temperature = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.05}
set_text_length = 420 #@param {type:"slider", min:0, max:1023, step:10}
set_nr_samples = 5 #@param {type:"slider", min:4, max:10, step:1}



print("please wait text is generated..." ) 

gpt2.generate(sess,
              run_name=give_run_name,
              length=set_text_length,
              temperature=set_temperature,
              prefix=paste_prefix_text,
              nsamples=set_nr_samples,
              batch_size=set_nr_samples
              )

please wait text is generated...
When you don't know what to do.

When you’re stuck in traffic.

When you’re bored at work.

When you’re tired.



These are all common complaints I hear from employees. I hear them all the time. “I’m stuck in traffic,” I hear you say. “I can’t get a rhythm here.” I hear it a million times. “I can’t get a rhythm here,” I hear it a million times. “I can’t get a rhythm here,” I hear it a million times. “I can’t get a rhythm here,” I hear it a million times. “I can’t get a rhythm here,” I hear it a million times. “I can’t get a rhythm here,” I hear it a million times. “I can’t get a rhythm here,” I hear it a million times. “I can’t get a rhythm here,” I hear it a million times. “I can’t get a rhythm here,” I hear it a million times. “I can’t get a rhythm here,” I hear it a million times. “I can’t get a rhythm here,” I hear it a million times. “I can’t get a rhythm here,” I hear it a million times. “Ican’t get a rhythm here,” I hear it a million times. “Ican